In [ ]:
import torch
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
drive.mount('/content/drive/')
import string
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import copy

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!ls drive/MyDrive/others/Data/Validation/
!ls drive/MyDrive/others/Test/hindi/test.csv

valid_Bangla.csv  valid_Hindi.csv  valid_Magahi.csv
drive/MyDrive/others/Test/hindi/test.csv


## Data Loading And Preprocessing

In [ ]:
train_df = pd.read_csv('drive/MyDrive/others/Data/Training/train_Hindi.csv')
train_df['sentence'] = train_df['sentence'].str.lower()
valid_df = pd.read_csv('drive/MyDrive/others/Data/Validation/valid_Hindi.csv')
valid_df['sentence'] = valid_df['sentence'].str.lower()
test_df = pd.read_csv('drive/MyDrive/others/Test/hindi/test.csv')
test_df['sentence'] = valid_df['sentence'].str.lower()

tokenizer = WhitespaceTokenizer()

punct = string.punctuation

def remove_punct_one(txt):
  for c in list(punct):
    txt = txt.replace(c, "")
  return txt

train_df['sentence'] = train_df['sentence'].apply(remove_punct_one)
valid_df['sentence'] = valid_df['sentence'].apply(remove_punct_one)
test_df['sentence'] = test_df['sentence'].apply(remove_punct_one)

### Vocab implementation on pre-processed dataset

In [ ]:
def prepare_datasets_and_vocab(data_frame):
    ## all the files for MEMD-ABSA
    sentences = list(map(lambda x: tokenizer.tokenize(x), data_frame['sentence']))
    vocab = build_vocab_from_iterator(sentences, specials=["<unk>"], min_freq=2)
    vocab.set_default_index(vocab['<unk>'])
    return vocab

In [ ]:
vocab = prepare_datasets_and_vocab(train_df)
print(f"Unique Words: {len(vocab)}")

Unique Words: 3332


In [ ]:
print('Senitment Labels:')
train_df['sentiment'].value_counts()

Senitment Labels:


positive    1898
negative     419
mix          113
neutral       77
Name: sentiment, dtype: int64

## Model Architecture
Embedding size: 100

BI-LSTM: 64*2

Fully Connected: 128

Fully Connected: 3

In [ ]:
class SentimentModel(torch.nn.Module):
    def __init__(self, vocab_size, emb_size):
        '''double embedding + lstm encoder + dot self attention'''
        super(SentimentModel, self).__init__()

        self.gen_embedding = torch.nn.Embedding(vocab_size, emb_size)
        self.dropout = torch.nn.Dropout(p=0.1)



        ## input size 400, output size 2 x 50
        self.norm1 = torch.nn.LayerNorm(emb_size)
        self.bilstm = torch.nn.LSTM(emb_size, 64,
                                    num_layers=2, batch_first=True, bidirectional=True)
        self.norm2 = torch.nn.LayerNorm(128)
        self.fc1 = torch.nn.Linear(128, 64)

        self.class_layer = torch.nn.Linear(64, 4)


    def forward(self, X):
        emb = self.dropout(self.gen_embedding(X))

        norm1 = self.norm1(emb)

        output, (h_n, c_n) =  self.bilstm(emb)
        # print(output.shape)
        # fc1 = self.fc1(F.relu(output[:, -1, :]))
        fc1 = self.fc1(self.norm2(output[:, -1, :]))
        classout = self.class_layer(fc1)
        return classout



In [ ]:
class SentDataset(Dataset):
  def __init__(self, data_df, tokenizer, vocab, maxlen=128):
    self.tokenizer = tokenizer
    self.vocab = vocab
    self.data_df = data_df
    self.maxlen = maxlen
    self.label_encoding = { # add more if more sentiment labels are present
        "negative": 0,
        "neutral": 1,
        "positive":2,
        "mix":3,
    }
    self.punct = string.punctuation
    self.encoded_data = []
    self._build()

  def _build(self):
    for sentence, label in self.data_df.values:
      enc_tokens = [0] * self.maxlen
      tokens = self.tokenizer.tokenize(sentence)
      if len(tokens)<=128:
        enc_tokens[:len(tokens)] = self.vocab(tokens)
      else:
        enc_tokens[:128] = self.vocab(tokens)[:128]
      lab = self.label_encoding[label]
      self.encoded_data.append([torch.tensor(enc_tokens),  torch.tensor(lab)])

  def __getitem__(self, index):
     return self.encoded_data[index]

  def __len__(self):
    return len(self.encoded_data)


In [ ]:
train_dataset = SentDataset(train_df, tokenizer, vocab)
valid_dataset = SentDataset(valid_df, tokenizer, vocab)
test_dataset = SentDataset(test_df, tokenizer, vocab)

In [ ]:
vals = dict(train_df['sentiment'].value_counts())
tot = sum(vals.values())
w_neg = 1/vals['negative']
w_neu = 1/vals['neutral']
w_pos = 1/vals['positive']
w_mix = 1/vals['mix']
sent_weights = torch.tensor([w_neg, w_neu, w_pos, w_mix], dtype=torch.float)
sent_weights.to(device)

tensor([0.0024, 0.0130, 0.0005, 0.0088], device='cuda:0')

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=128)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
model = SentimentModel(len(vocab), 100)
loss_fn = torch.nn.CrossEntropyLoss(weight=sent_weights)
loss_fn.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)#, weight_decay=0.001)
best_model = None
model.to(device)


SentimentModel(
  (gen_embedding): Embedding(3332, 100)
  (dropout): Dropout(p=0.1, inplace=False)
  (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
  (bilstm): LSTM(100, 64, num_layers=2, batch_first=True, bidirectional=True)
  (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (class_layer): Linear(in_features=64, out_features=4, bias=True)
)

In [ ]:
def eval_loop(pred_model, dataloader):
  dload = tqdm(dataloader)
  preds = []
  golds = []
  pred_model.to(device)
  pred_model.eval()
  with torch.no_grad():
    for batch in dload:
      x, y = batch
      pred_out = pred_model(x.to(device))
      predictions = pred_out.argmax(dim=-1).tolist()
      gold_labs = y.tolist()
      for i in range(len(predictions)):
        preds.append(predictions[i])
        golds.append(gold_labs[i])

    precise_macro = precision_score(golds, preds, average='macro')#, zero_division=0)
    recall_macro = recall_score(golds, preds, average='macro')#, zero_division=0)
    f1_macro = f1_score(golds, preds, average='macro')#, zero_division=0)

  return preds, {
      'macro_precision': precise_macro,
      'macro_recall': recall_macro,
      'macro_f1': f1_macro
  }

In [ ]:
N_EPOCHS = 80
best_f1 = -1
for epoch in range(N_EPOCHS):
  train_epoch_loss = 0
  model.to(device)
  model.train()
  train_ite = tqdm(train_dataloader)
  for batch in train_ite:
    x, l = batch
    x = x.to(device)
    l = l.to(device)
    output = model(x)

    loss = loss_fn(output, l)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    train_ite.set_postfix({'training loss': loss.item()})
    train_epoch_loss += loss.item()

  validation_epoch_loss = 0
  valid_ite = tqdm(valid_dataloader)
  preds = []
  golds = []
  model.eval()
  with torch.no_grad():
    for batch in valid_ite:
      x, y = batch
      pred_out = model(x.to(device))
      preds += pred_out.argmax(dim=-1).tolist()
      golds += y.tolist()
      val_loss = loss_fn(pred_out.to(device),y.to(device))
      valid_ite.set_postfix({'validation loss': val_loss.item()})
      validation_epoch_loss = val_loss.item()
    print()
    print('--'*20, ' Validation Scores ', '--'*20)
    precise_macro = precision_score(golds, preds, average='macro', zero_division=0)
    recall_macro = recall_score(golds, preds, average='macro', zero_division=0)
    f1_macro = f1_score(golds, preds, average='macro', zero_division=0)
    if f1_macro > best_f1:
      best_model = copy.deepcopy(model)
      best_f1 = f1_macro
    print(f'EPOCH: {epoch}')
    print(f'avg training loss: {(train_epoch_loss/len(train_dataset))}, avg validation loss:{(validation_epoch_loss/len(valid_dataset))}')
    print(f'precision:{precise_macro}, recall:{recall_macro}, f1:{f1_macro}')
    # if best_f1<f1_macro:
    print('--'*50)



100%|██████████| 17/17 [00:00<00:00, 268.33it/s, validation loss=1.4]



----------------------------------------  Validation Scores  ----------------------------------------
EPOCH: 0
avg training loss: 0.011946265470187503, avg validation loss:0.002605934568497328
precision:0.007434944237918215, recall:0.25, f1:0.014440433212996389
----------------------------------------------------------------------------------------------------


100%|██████████| 17/17 [00:00<00:00, 252.25it/s, validation loss=1.48]



----------------------------------------  Validation Scores  ----------------------------------------
EPOCH: 1
avg training loss: 0.011144040826311328, avg validation loss:0.0027590159146759145
precision:0.007434944237918215, recall:0.25, f1:0.014440433212996389
----------------------------------------------------------------------------------------------------


100%|██████████| 17/17 [00:00<00:00, 254.30it/s, validation loss=1.36]


...


----------------------------------------  Validation Scores  ----------------------------------------
EPOCH: 77
avg training loss: 0.0009223391403346409, avg validation loss:0.005733383189346711
precision:0.35607755581668626, recall:0.38424293611793614, f1:0.36502641153803944
----------------------------------------------------------------------------------------------------


100%|██████████| 17/17 [00:00<00:00, 277.35it/s, validation loss=3.63]



----------------------------------------  Validation Scores  ----------------------------------------
EPOCH: 78
avg training loss: 0.000611950755178762, avg validation loss:0.006744942700552674
precision:0.36945807154413446, recall:0.4048844867594868, f1:0.3812047376558103
----------------------------------------------------------------------------------------------------


100%|██████████| 17/17 [00:00<00:00, 211.88it/s, validation loss=3.69]



----------------------------------------  Validation Scores  ----------------------------------------
EPOCH: 79
avg training loss: 0.0007603637663120761, avg validation loss:0.006850207605326486
precision:0.3645596786970246, recall:0.42534995222495225, f1:0.38030745207537864
----------------------------------------------------------------------------------------------------


## TESTING

In [ ]:
!ls drive/MyDrive/others/Test/Trac1_test/

bangla_test.csv        combine_test.csv     hin_test_.gsheet	mag_test.gsheet
bangla_test.gsheet     combine_test.gsheet  hin_test_w_ids.csv	mag_test_w_ids.csv
bangla_test_w_ids.csv  hin_test_.csv	    mag_test.csv


In [ ]:
test_df = pd.read_csv('drive/MyDrive/others/Test/Trac1_test/hin_test_w_ids.csv')
test_df['sentence'] = valid_df['sentence'].str.lower()


test_df['sentence'] = test_df['sentence'].apply(remove_punct_one)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
preds, stats = eval_loop(best_model, test_dataloader)

100%|██████████| 34/34 [00:00<00:00, 474.84it/s]


In [ ]:
stats

{'macro_precision': 0.25320348173515983,
 'macro_recall': 0.244972692304153,
 'macro_f1': 0.2416986809554397}

In [ ]:
key_to_val = {v:k for k,v in test_dataset.label_encoding.items()}
decoded_pred = [key_to_val[pred] for pred in preds]

In [ ]:
test_df['pred'] = decoded_pred

In [ ]:
correct = test_df[test_df['sentiment'] == test_df['pred']]

In [ ]:
correct['sentiment'].value_counts()


positive    246
negative     26
mix           2
Name: sentiment, dtype: int64